<div style="
    border: 2px solid #4CAF50; 
    padding: 15px; 
    background-color: #f4f4f4; 
    border-radius: 10px; 
    align-items: center;">

<h1 style="margin: 0; color: #4CAF50;">Neural Networks: Convolutional Neural Networks (1)</h1>
<h2 style="margin: 5px 0; color: #555;">DSAI</h2>
<h3 style="margin: 5px 0; color: #555;">Jakob Eggl</h3>

<div style="flex-shrink: 0;">
    <img src="https://www.htl-grieskirchen.at/wp/wp-content/uploads/2022/11/logo_bildschirm-1024x503.png" alt="Logo" style="width: 250px; height: auto;"/>
</div>
<p1> © 2025/26 Jakob Eggl. Nutzung oder Verbreitung nur mit ausdrücklicher Genehmigung des Autors.</p1>
</div>
<div style="flex: 1;">
</div>   

Nachdem wir bisher nur von `Linear`-Layers gesprochen haben, möchten wir uns jetzt auf eine **neue Architektur** fokusieren. Diese ist besonders geeignet für Bilder und hat eine gravierend andere funktionsweise als bisherige Layers.

Die Rede ist von sogenannten **Convolution**-Layers, welche dann die sogenannten Convolutional Neural Networks (CNN's) bilden.

In diesem Notebook wollen wir:
1) Zuerst die Convolution-Operation kennenlernen und deren Anwendung in der klassischen Bildbearbeitung betrachten bevor wir
2) Uns das Convolution-Layer in PyTorch ansehen und uns am Schluss
3) Über die weiteren damit verbundenen Layers wie Pooling befassen.

**Hinweis:** *Convolution* ist der englische Begriff für *Faltung*.

**Hinweis:** Die Faltung ist eine mathematische Operation, welche [hier](https://de.wikipedia.org/wiki/Faltung_(Mathematik)) nachgelesen werden *kann*.

## Die Convolution-Operation

Zu Beginn wollen wir uns die Faltungsoperation auf Bildern ansehen.

In diesem Notebook haben wir dabei immer folgendes Setup:
* Wir haben ein Bild, sprich mathematisch gesehen einen $(3, \text{Hoehe}, \text{Breite})$ Tensor, also dreimal eine Matrix der Größe $\text{Hoehe} \times \text{Breite}$.
* Wir haben einen Filter $K$ (auch Kernel genannt), mathematisch gesehen ist das auch ein Tensor, der Einfachkeit halber (und das trifft auch meistens zu) denken wir einfach an eine Matrix.
* Wir wollen den Filter auf unser Bild anwenden und dabei ein neues Bild, also eine neue Matrix generieren.

Betrachten wir hier zuerst mal ein paar Beispiele.

<img src="../resources/Mario_Convolution.png" width="1000"/>

(von https://www.youtube.com/watch?v=KuXjwB4LzSA)

<img src="../resources/Kirby_Convolution.jpg" width="1000"/>

(von https://www.youtube.com/watch?v=KuXjwB4LzSA)

Hier ist bei beiden Bildern rechts oben bzw. auch links das große Bild das Ausgangsbild. Rechts oben (blau umrahmt) ist der Filter zu sehen.

Das Ergebnis der Faltungsoperation (Convolution) ist unten rechts zu sehen:
Im Super-Mario Bild ist das ein Unschärfe Filter, im zweiten Bild mehr oder weniger das Gegenteil, sprich ein Filter zum Schärfen des Bildes.

**Aber was passiert da genau?**

Wir betrachten dazu ein **schwarz-weiß Bild**, also eine Matrix als unseren Input.


In unserem Fall ist das die **Matrix**:
$$X=\begin{pmatrix}
    0 & 0 & 0 & 0 & 0 & 0\\
    0 & 1.0 & 0 & 0 & 0.4 & 0\\
    0 & 0 & 0 & 0.4 & 0 & 0\\
    0 & 0 & 0 & 0 & 0.4 & 0\\
    0 & 0 & 0 & 0.4 & 0 & 0\\
    0 & 0 & 0 & 0 & 0.4 & 0
\end{pmatrix}$$

Dazu wollen wir als **Kernel (=Filter)** folgende Matrix verwenden

$$K = \begin{pmatrix}
    0 & 0 & 0.5 \\
    0 & 0.5 & 0 \\
    0 & 0 & 0.5
\end{pmatrix}$$

Bei der Faltung wird nun der Filter über den Input $X$ gelegt, und das in jeder möglichen Kombination.

Das sieht in unserem Fall dann so aus für den ersten Eintrag des Ergebnisses.

<img src="../resources/Convolution_Concept1.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

**Was ist hier passiert?** Wir haben den Filter (rot) über unser Bild (blau) gelegt und danach einfach Elementweise Matrix-Multipliziert und das Ergebnis zusammen gezählt.

Dies wiederholen wir jetzt auch für die anderen Möglichkeiten. Also wir shiften unseren Kern um 1 Spalte nach rechts.

<img src="../resources/Convolution_Concept2.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

Dies wiederholen wir jetzt solange, bis wir alle Möglichkeiten durch haben.

<img src="../resources/Convolution_Concept3.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Concept4.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Concept5.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Concept6.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Concept7.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

**Wichtig:** Wie wir sehen ist also die Faltung quasi (mathematisch sehr ungenau) wie ein 2D-Skalarprodukt.

Somit können wir auch erklären, warum beim Super-Mario Bild vorher ein unscharfer Output erzeugt wird. Der Grund ist, weil einfach der neue Pixel-Wert der Durchschnitt von allen umliegenden Pixelwerten ist.

**Wichtig:** Beim Super-Mario und Kirby Bild ist die Faltung jeweils auf jeden Channel ausgeführt worden. Dabei wurde jedes mal der gleiche Filter (Kernel) verwendet.

Wir wollen nun ein paar weitere Filter ausprobieren bei eigenen Bildern. Dazu nutzen wir die `cv2` Bibliothek. Das geht (falls noch nicht vorhanden mit dem `pip install opencv-python` Command).

In [ ]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
image_path = os.path.join("..", "resources", "Chemnitz_Hauptplatz.jpg")
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

fig, ax = plt.subplots(figsize=(3, 6))
ax.imshow(image, cmap="gray", vmin=0, vmax=255)

ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])

plt.show()

Nun definieren wir unsere Kernel. Beispiele sind zum Beispiel die Sobel Filter.

In [ ]:
sobel_x = np.array([
    [ -1,  0,  1],
    [ -2,  0,  2],
    [ -1,  0,  1]
])
sobel_y = np.array([
    [ -1, -2, -1],
    [  0,  0,  0],
    [  1,  2,  1]
])


mean_blur = 1/400 * np.ones((20, 20))

edge_detection = sobel_x + sobel_y

gaussian_blur = 1/36 * np.array([
    [1,4,1],
    [4,16,4],
    [1,4,1]
]) 

filter = edge_detection

filtered_image = cv2.filter2D(image, -1, filter)


fig, ax = plt.subplots(figsize=(3, 6))
ax.imshow(filtered_image, cmap="gray", vmin=0, vmax=255)
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
plt.show()

> **Übung:** Suche im Internet nach dem **Edge-Detection**, **Mean-Blurr**, **Gaussian-Blurr** und einem Schärfefilter und implementiere diese oben. 

**Hinweis:** Der Filter hat normalerweise nur positive Einträge (falls negativ muss man sich überlegen, wie das interpretiert werden soll) und diese Einträge sollen in Summe 1 Ergeben, sodass die (Pixel)Werte des Outputs im gleichen Bereich bleiben.

> **Übung:** Was ändert sich beim Output neben den eigentlichen Pixel Werten noch? *Tipp:* Betrachte nochmal die genaue Berechnung im vorigen Beispiel, welches in vielen Schritten detailiert gezeigt wurde.

Wie wir bereits bemerkt haben, ist die Output Matrix etwas kleiner als der Input. Die genaue Größe kann folgendermaßen berechnet werden:

$$\begin{align*}
    X_{\text{new}} &= \left\lfloor \frac{X-K_x+2P_x}{S_x} + 1 \right\rfloor  \\
    Y_{\text{new}} &= \left\lfloor \frac{Y-K_y+2P_y}{S_y} + 1 \right\rfloor
\end{align*}$$

Dabei ist:

* $X_{\text{new}}, Y_{\text{new}}$ die Größe der neuen Matrix
* $K_x$, $K_y$ die Größe des Kernels ($K_x$ ist die Anzahl der Spalten)
* $P_x, P_y$ steht für das Padding. Dieser Parameter erlaubt uns, einen gleich großen Output wie vorher der Input zu haben. Er beschreibt wie viele Pixel wir rund um unsere Matrix noch hinzufügen. Sprich $P_x=1$ heißt, wir fügen an der linken und an der rechten Seite noch eine Spalte hinzu. Als Wert wird dabei normalerweise die $0$ verwendet ("**Zero-Padding**"), es gibt aber auch andere Möglichkeiten.
* $S_x$, $S_y$ steht für den Stride. Dieser steht für die Anzahl an Pixel, die wir jedes mal nach $x$ bzw. nach $y$ "rutschen". Standard ist $1$, also wir bewegen uns immer nur 1 Pixel.

Sehr empfehlenswert ist hier dieses [GitHub Repository](https://github.com/vdumoulin/conv_arithmetic), da es sehr viele Visualisierungen beinhaltet zu den einzelnen Convolution Typen.

**Visualisierung Stride=3**

<img src="../resources/Convolution_Stride3_1.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Stride3_2.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Stride3_3.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Stride3_4.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Stride3_5.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

**Visualisierung Padding=1**

<img src="../resources/Convolution_Zero_Padding_1.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Convolution_Zero_Padding_2.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

## Der CNN-Layer

Nachdem wir nun Bescheid wissen, wie die Faltung auf Bildern allgemein funktioniert, betrachten wir nun das CNN-Layer in PyTorch.

Prinzipiell ist es immer gut, einen Blick in die Dokumentation zu werfen. Deswegen werden wir zuerst [hier](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) kurz schauen. Wir betrachten dabei das `nn.Conv2d()`-Layer, da wir es hauptsächlich auf Bildern (2D) anwenden werden.

`class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, ..., padding_mode='zeros')`

Dies ist der (wichtige Teil vom) Konstruktor des `Conv2d` Layers. Wir gehen nun die Parameter durch:

* `in_channels`: Die Anzahl der Input Channels. Für ein RGB Bild ist dies 3.
* `out_channels`: Anzahl der Kernels, die wir durch das Netzwerk schicken. Dies ist dann die Anzahl der **Output Channels** und somit auch gleich `in_channels`, falls danach eine weitere `nn.Conv2d` Schicht kommt.
* `kernel_size`: Größe des Kernels (in Tupelform, also $(3,5)$ oder in Integerform, also $3$ für einen quadratischen Kernel (in diesem Fall $3\times 3$))
* `stride`: Größe des Strides (erneut entweder Tupelform oder Integer bei gleichem Stride)
* `padding`: Größe des Paddings (Tupel oder Integer)
* `padding_mode`: Bestimmt die Art des Paddings. Übergeben wird ein String: 'zeros', 'reflect', 'replicate' oder 'circular'. Standard ist 'zeros'. 

> **Übung:** Was sind nun die Parameter die gelernt werden sollen vom Netzwerk?

Wir wollen also die Werte des Filters (Kernel) lernen.

> **Übung:** Hat ein CNN mehr oder weniger Parameter als ein Fully-Connected Neural Network?

Wir visualisieren nochmal kurz die Parameter des Netzwerkes.

<img src="../resources/CNN_Input_1.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/CNN_Input_2.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

Wir sehen also, dass zum Beispiel für einen RGB Input der Kernel dann auch einfach 3 Channels hat (3 **verschiedene** Matrizen). Dabei wird jeder Layer des Kernels auf das entsprechende Layer im Bild angewendet. Am Ende werden alle Schichten zu **einem Channel** zusammen **addiert**.

Beim zweiten Bild ist nun die Anzahl an `out_channels` auf 2 gesetzt. Das heißt wir haben 2 **verschiedene** Kernels mit je 3 Channels. 

**Was ist nun das Ziel von den Kernels?**

Die Kernels versuchen im Lernprozess Werte in die einzelnen Einträge zu schreiben, sodass wir aus unseren Trainingsbildern Schritt für Schritt möglichst gute Informationen extrahieren können.

Prinzipiell haben wir dadurch schon verstanden wie ein CNN-Layer in PyTorch funktioniert. Was gibt es jetzt noch zu beachten?

* Wir müssen nach jedem CNN-Layer berechnen, wie groß unser Ergebnis-Bild ist. Der Grund ist, dass wir nach unseren CNN-Layers eine fixe Output Größe brauchen (siehe nächster Punkt).
* Der Output von einem Neuronalen Netzwerk, welches CNN-Schichten beinhaltet ist vielfältig:
    * In den meisten Fällen wechseln wir nach einigen CNN-Layern zu einem Fully-Connected Neuronal Network, welches später die Klassifikation bzw. Regression basierend auf den Ergebnissen der Faltung(en) erledigt. Dabei müssen wir wissen, wie groß der Output nach dem letzten CNN-Layer ist. Ist dieser dann zum Beispiel ein $3\times 3$-Bild, dann würden wir dieses `flatten()` und erhalten einen $9$-Vektor.
    * Es gibt aber auch Fälle (zum Beispiel bei unserer Image Inpainting Challenge), bei der wir als Output wirklich Bilder wollen, sprich wir beenden unser Netzwerk auch mit einem CNN-Layer. Auch hier müssen wir sicherstellen, dass dieser Output dann zum Beispiel genauso groß ist wie der Input.

> **Übung:** Wie können wir das mit CNN's realisieren, dass am Ende unser Bild genauso groß ist wie vorher?

Am Schluss wollen wir uns noch einem weiteren, sehr ähnlichen, Konzept widmen. Die Rede ist von den sogenannten **Pooling-Layers**.

## Pooling

Die Idee vom **Pooling** ist, dass wir unser Bild *downsamplen*, sprich ein kleineres Bild produzieren. Dabei gibt es mehrere Möglichkeiten, wie wir dieses Downsampling realisieren können:
* **Average-Pooling:** Wir nehmen den Mittelwert von $k\times k$ Werten
* **Max Pooling:** Wir nehmen den Maximalwert von $k\times k$ Werten (wird meistens verwendet)

Wir visualisieren kurz den Effekt von Max-Pooling, in diesem Fall mit $k=2$. 

<img src="../resources/Max_Pooling_1.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Max_Pooling_2.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Max_Pooling_3.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Max_Pooling_4.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

<img src="../resources/Max_Pooling_5.jpg" width="1000"/>

(von Dr. Andreas Schörgenhumer; Hands-On AI1 WS2021)

**Wichtig:** Ein Pooling Layer beinhaltet also keine Parameter, er reduziert nur unsere Datengröße deutlich und führt somit auch natürlich zu einem (großen) Informationsverlust.

In PyTorch können wir so ein Verhalten auch ganz einfach mit einem Pooling Layer erreichen, welches wir mit `nn.MaxPool2d()` ganz einfach hinzufügen können. Die Dokumentation finden wir [hier](https://docs.pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html).

Betrachten wir kurz (den wichtigsten Teil davon) den Konstruktor.

`class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, ...)`

Der einzige Wert im Konstruktor, der übergeben werden muss ist `kernel_size`, erneut als Tupel oder Integer. Als `stride` ist standardmäßig `None` übergeben, was bedeutet, dass wir pro Schritt den "Filter" um die `kernel_size` verschieben. Ebenso ist kein standardmäßig kein Padding vorgesehen.

**Hinweis:** Die Größe der Daten kann nach einem Pooling Layer mit der gleichen Formel wie vorher berechnet werden. In vielen Fällen (`stride=kernel_size` und ohne Padding) ist das (bis auf das Verhalten am Rand) die Berechnung natürlich ohne Formel auch sehr leicht.

---

Damit haben wir die Grundlagen eines CNN-Layers verstanden. Wir werden im nächsten Notebook ein großes Beispiel machen, welches uns die Details und die Funktionen in einem praktischen Setting nochmal näher bringt.